In [4]:
# IMPORTS
import geemap
import ee
ee.Initialize()

from amigocloud import AmigoCloud

from shapely import wkb
import geopandas
import pandas as pd

import ipywidgets as widgets
from datetime import datetime

In [5]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
Map = geemap.Map(basemap="SATELLITE")

# variables globales
COD_CA = 1672
# dataframes
LOTES = None
CANERO = None
CALIDAD_SIEMBRA=None
CALIDAD_SOCA=None
PROPIEDAD=None

In [7]:
PROPIEDAD

In [ ]:
##################################
# FUNCIONES GENERALES
##################################

# funcion para conversion de wkb a poligonos
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

def extraer_lotes_credito(amigo_id_prop):
    # consulta sql para extraer los lotes de un credito
    query_credito = {'query': 'select * from dataset_282975 where (estado_de_siembra=\'CAÑA NACIDA\' or estado_de_siembra=\'CAÑA NO NACIDA\') and prop_ref_lote = \'{prop_id}\''.format(prop_id=amigo_id_prop)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_credito)
    credito_data = credito_select['data']
    
    # carga de datos seleccionados a DadaFrame
    df = pd.DataFrame(credito_data)
    df['geometria'] = df['geometria'].apply(convertir_wkb)
    # DataFrame a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    # convercion a FeatureColleccion
    ee_lotes = ee.FeatureCollection(data.__geo_interface__)
    return ee_lotes

def buscar_props_credito(cod_ca):
    query_credito = {'query': 'SELECT \
                     propiedad.propiedad PROPIEDAD, \
                     sum(lote.superficie) SUPERFICIE, \
                     propiedad.amigo_id \
                     FROM dataset_282969 credito \
                     INNER JOIN dataset_282968 propiedad ON credito.amigo_id=propiedad.cre_ref_prop\
                     INNER JOIN dataset_282975 lote ON propiedad.amigo_id=lote.prop_ref_lote\
                     where cast(SPLIT_PART(credito.canhero, \' / \', 1) as integer) = {cod}\
                     and (lote.estado_de_siembra=\'CAÑA NACIDA\' or lote.estado_de_siembra=\'CAÑA NO NACIDA\') \
                     group by propiedad.propiedad, propiedad.amigo_id'.format(cod=cod_ca)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_credito)
    credito_data = credito_select['data']
    return credito_data

def buscar_datos_cañero(cod_ca):
    query_credito = {'query': 'select canhero, solicitado, total_1ra_insp, respon_1ra_insp, muestreo, glifosato, nivelacion_1, drenaje,\
                                romeplow, rastra, subsolado, rotacion, arado, ferti_base, enraizadores, insecticidas, fungicidas, preemergente \
                                from dataset_282969 \
                                where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = {cod}'.format(cod=cod_ca)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_credito)
    credito_data = credito_select['data']
    return credito_data
    
def buscar_canhero(gui):
    global CANERO
    global CALIDAD_SIEMBRA
    out.clear_output()
    with out:
        display('CARGANDO DATOS...')
    
    canhero = buscar_datos_cañero(cod_ca.value)
    CANERO = canhero
    df = pd.DataFrame(canhero)
    out_canhero.clear_output()
    with out_canhero:
        display(df)
    
    if canhero != []:
        propiedades = buscar_props_credito(cod_ca.value)
        
        lista = []
        for i in propiedades:
            texto = i['propiedad'] + '//' + str(i['superficie']) + '//' + i['amigo_id']
            lista.append(texto)
        lista_props.options=lista
    
        df = pd.DataFrame(propiedades)
    else:
        out_canhero.clear_output()
        with out_canhero:
            display('ESTE CAÑERO NO TIENE PROPIEDADES ASIGNADAS')
    #cargar_calidad_siembra()
    datos = cargar_calidad_siembra_por_prop()
    CALIDAD_SIEMBRA = datos
    if(datos != []):
        df = pd.DataFrame(datos)
        out_calidad_siembra.clear_output()
        with out_calidad_siembra:
            display(df)
    else:
        with out_calidad_siembra:
            display('SIN DATOS DE CALIDAD DE SIEMBRA PARA LA PROPIEDAD...!!!')

def buscar_props():
    cod_canhero = cod_prop.value
    props = buscar_props_credito(cod_canhero)

def get_fechas(coll):
    def extraer_fecha(img):
        return ee.Image(img).date().format('dd/MM/YYYY')
    return coll.toList(coll.size(),0).map(extraer_fecha)    
    
def buscar_imagenes(lotes):
    coleccion = ee.ImageCollection('COPERNICUS/S2_SR')\
                .filterBounds(lotes.geometry())\
                .filterDate(fecha_inicio.value.strftime('%Y-%m-%d'), fecha_final.value.strftime('%Y-%m-%d'))\
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', nubocidad.value)
    
    # extraer IDs de la coleccion
    ids = coleccion.reduceColumns(ee.Reducer.toList(), ['system:index']).get('list').getInfo()
    # extraer fechas
    fechas = get_fechas(coleccion).getInfo()
    # tabla de datos
    #dic = {'FECHA':fechas, 'ID':ids}
    
    #DF_IMGS = pd.DataFrame(dic)
    lista=[]
    contador=0
    for i in ids:
        texto = fechas[contador] + '//' + i
        lista.append(texto)
        contador=contador+1
    lista_imgs.options=lista
    
def cargar_lotes(gui):
    global CALIDAD_SIEMBRA
    global CALIDAD_SOCA
    global PROPIEDAD
    
    PROPIEDAD = ee.FeatureCollection('projects/ee-bismarksr17/assets/CATASTRO220430')\
                .filter(ee.Filter.eq('unidad_01', int(lista_props.value.split('//')[0].split(' / ')[0])))
    
    out.clear_output()
    with out:
        display('CARGANDO DATOS...')
    global LOTES
    amigo_id = gui['owner'].value.split('//')[2]
    ee_lotes = extraer_lotes_credito(amigo_id)
    LOTES = ee_lotes
    
    lista_lotes = ee_lotes.toList(ee_lotes.size(), 0).getInfo()
    lista = []
    for l in lista_lotes:
        lista.append(l['properties'])
    df = pd.DataFrame(lista)
    df = df.drop(['amigo_id', 'estado_de_siembra', 'fecha_actualizacion', 'fecha_registro', 'fecha_siembra', 'obs_2','prop_ref_lote','variedad_sembrada'], axis=1)
    out_lotes.clear_output()
    with out_lotes:
        display(df)
    
    Map.addLayer(PROPIEDAD.style(**vis_params_propiedad), {},'PROPIEDAD')
    
    Map.centerObject(ee_lotes, 14)
    Map.addLayer(ee_lotes.style(**vis_params_caña), {},'LOTES')
    
    buscar_imagenes(ee_lotes)
    
    # datos de calidad de siembra
    datos = cargar_calidad_siembra_por_prop()
    CALIDAD_SIEMBRA = datos
    if(datos != []):
        df = pd.DataFrame(datos)
        out_calidad_siembra.clear_output()
        with out_calidad_siembra:
            display(df)
    else:
        out_calidad_siembra.clear_output()
        with out_calidad_siembra:
            display('SIN DATOS DE CALIDAD DE SIEMBRA PARA LA PROPIEDAD...!!!')
    
    # datos de calidad de soca
    datos = buscar_calidad_soca_por_prop()
    CALIDAD_SOCA = datos
    if(datos != []):
        df = pd.DataFrame(datos)
        out_calidad_soca.clear_output()
        with out_calidad_soca:
            display(df)
    else:
        out_calidad_soca.clear_output()
        with out_calidad_soca:
            display('SIN DATOS DE CALIDAD DE SOCA PARA LA PROPIEDAD...!!!')
            
    out.clear_output()
    with out:
        display('LISTO...!!!')

def cargar_img(gui):
    
    id_imagen = gui['owner'].value.split('//')[1]

    # carga la imagen con las bandas de interes
    img = ee.Image('COPERNICUS/S2_SR/'+id_imagen)\
            .multiply(0.0001)\
            .select(['B2','B3','B4','B8'], ['BLUE','GREEN','RED','NIR'])
    Map.addLayer(img, viz_img, 'Imagen Sat')
    Map.addLayer(PROPIEDAD.style(**vis_params_propiedad), {},'PROPIEDAD')
    Map.addLayer(LOTES.style(**vis_params_caña), {}, 'LOTES')
    
    out.clear_output()
    with out:
        display('LISTO...!!!')

'''
def buscar_calidad_siembra(cod_ca, cod_prop):
    query_credito = {'query': 'select muest_suelo, glifosato, nivelacion, drenaje, romeplow, rastra, subsolado, arado, rotacion, fert_base, enraizadores, fungicidas, pre_emergente \
                                from dataset_273464 where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = {cod_ca} and cast(SPLIT_PART(propiedad, \' / \', 1) as integer) = {cod_prop}'.format(cod_ca=cod_ca, cod_prop=cod_prop)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30860/sql', query_credito)
    credito_data = credito_select['data']
    return credito_data
'''

def buscar_calidad_siembra_por_prop(cod_prop):
    query_credito = {'query': 'select cast(SPLIT_PART(canhero, \' / \', 1) as integer) cod_ca, SPLIT_PART(canhero, \' / \', 2) nom_ca,  responsable, muest_suelo, glifosato, nivelacion, drenaje, romeplow, rastra, subsolado, rotacion, arado, fert_base, enraizadores, insecticidas, fungicidas, pre_emergente \
                                from dataset_273464 where cast(SPLIT_PART(propiedad, \' / \', 1) as integer) = {cod_prop}'.format(cod_prop=cod_prop)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30860/sql', query_credito)
    credito_data = credito_select['data']
    return credito_data
'''
def cargar_calidad_siembra():
    ca = cod_ca.value
    prop = int(lista_props.value.split('//')[0].split(' / ')[0])
    datos = buscar_calidad_siembra(ca, prop)
    if(datos != []):
        with out_canhero:
            display(datos)
    else:
        with out_canhero:
            display('SIN DATOS DE CALIDAD DE SIEMBRA PARA EL CAÑERO Y LA PROPIEDAD...!!!')
'''

def cargar_calidad_siembra_por_prop():
    prop = int(lista_props.value.split('//')[0].split(' / ')[0])
    datos = buscar_calidad_siembra_por_prop(prop)
    return datos

def buscar_calidad_soca_por_prop():
    prop = int(lista_props.value.split('//')[0].split(' / ')[0])
    query_credito = {'query': 'select cast(SPLIT_PART(canhero, \' / \', 1) as integer) cod_ca, SPLIT_PART(canhero, \' / \', 2) nom_ca, responsable, entera_fresca, entera_quemada, picada_fresca, picada_quemada, quema, hilerado, permanente, subsolado, fertilizado, pre_emergente, post_emergente \
                                from dataset_272975 where cast(SPLIT_PART(propiedad, \' / \', 1) as integer)={cod_prop}'.format(cod_prop=prop)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query_credito)
    credito_data = credito_select['data']
    return credito_data



In [ ]:
##################################
# PARAMETROS DE VISUALIZACION
##################################
vis_params_caña = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

vis_params_propiedad = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

viz_img = {
    'bands':['RED','GREEN','BLUE'],
    'min':0.0,
    'max':0.5,
    'fillColorOpacity': '0'
}

In [ ]:
##################################
# CONTROLES
##################################

cod_ca = widgets.BoundedIntText(
    value=1672,
    min=0,
    max=99999999999999999,
    step=1,
    disabled=False
)

btn_buscar_canhero = widgets.Button(
    description='Buscar Cañero',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Buscar',
)
btn_buscar_canhero.on_click(buscar_canhero)

lista_props = widgets.Dropdown(
    options=['-'],
    value='-'
)
lista_props.observe(cargar_lotes, names='index')

# datepiker para fecha inicio
fecha_inicio = widgets.DatePicker(
    value=datetime(2022, 1, 1, 10, 15, 00, 00000),
    disabled=False
)

# datepiker para fecha fin
fecha_final = widgets.DatePicker(
    value=datetime.now(),
    disabled=False
)

nubocidad = widgets.BoundedIntText(
    value=30,
    min=0,
    max=100,
    step=10,
    disabled=False
)

lista_imgs = widgets.Dropdown(
    options=['-'],
    value='-'
)
lista_imgs.observe(cargar_img, names='index')


In [ ]:
##################################
# FORMULARIO
##################################

form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    width='100%'
)

form_items = [
    widgets.Box([widgets.Label(value='COD. CAÑERO:'), cod_ca], layout=form_item_layout),
    widgets.Box([widgets.Label(value='BUSCAR CAÑERO:'), btn_buscar_canhero], layout=form_item_layout),
    widgets.Box([widgets.Label(value='PROPIEDADES:'), lista_props], layout=form_item_layout),
    widgets.Box([widgets.Label(value='FECHA INICIO:'), fecha_inicio], layout=form_item_layout),
    widgets.Box([widgets.Label(value='FECHA FINAL:'), fecha_final], layout=form_item_layout),
    widgets.Box([widgets.Label(value='NUBOCIDAD:'), nubocidad], layout=form_item_layout),
    widgets.Box([widgets.Label(value='IMAGENES:'), lista_imgs], layout=form_item_layout)
]

form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%'
))
form

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
out

CREDITO

In [ ]:
out_canhero = widgets.Output(layout={'border': '1px solid black'})
out_canhero

CALIDAD SIEMBRA

In [ ]:
out_calidad_siembra = widgets.Output(layout={'border': '1px solid black'})
out_calidad_siembra

CALIDAD SOCA

In [ ]:
out_calidad_soca = widgets.Output(layout={'border': '1px solid black'})
out_calidad_soca

In [ ]:
out_props = widgets.Output(layout={'border': '1px solid black'})
out_props

In [ ]:
Map

In [ ]:
out_lotes = widgets.Output(layout={'border': '1px solid black'})
out_lotes

In [ ]:
out_lotes = widgets.Output(layout={'border': '1px solid black'})
out_lotes

In [ ]:
df2 = pd.DataFrame(CALIDAD_SIEMBRA)
df2